<a href="https://colab.research.google.com/github/djbirdsall/QA_Squad/blob/main/transformers_doc/en/question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Transformers installation
# pip install accelerate transformers[torch] transformers datasets evaluate keras tensorflow torch PyTorch
# pip install datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# pip install git+https://github.com/huggingface/transformers.git

# Question answering

In [19]:
# from huggingface_hub import notebook_login

# notebook_login()

## Load SQuAD dataset

In [147]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)

## Preprocess

In [148]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("djbirdsall/qa_model_tutorial")

In [149]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [150]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [151]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

## Train

In [152]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [153]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("djbirdsall/qa_model_tutorial")

Some layers from the model checkpoint at djbirdsall/qa_model_tutorial were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_419']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at djbirdsall/qa_model_tutorial and are newly initialized: ['dropout_499']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [154]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [155]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [156]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="djbirdsall/qa_model_tutorial",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/djbirdsall/qa_model_tutorial is already a clone of https://huggingface.co/djbirdsall/qa_model_tutorial. Make sure you pull the latest changes with `repo.git_pull()`.


In [157]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=10, callbacks=[callback])

Epoch 1/10
250/250 [==============================] - 90s 297ms/step - loss: 0.5566 - val_loss: 0.4185
Epoch 2/10
250/250 [==============================] - 70s 280ms/step - loss: 0.3153 - val_loss: 0.4077
Epoch 3/10
250/250 [==============================] - 62s 249ms/step - loss: 0.2534 - val_loss: 0.4077
Epoch 4/10
250/250 [==============================] - 62s 249ms/step - loss: 0.2599 - val_loss: 0.4077
Epoch 5/10
250/250 [==============================] - 63s 251ms/step - loss: 0.2598 - val_loss: 0.4077
Epoch 6/10
250/250 [==============================] - 63s 251ms/step - loss: 0.2592 - val_loss: 0.4077
Epoch 7/10
250/250 [==============================] - 63s 252ms/step - loss: 0.2527 - val_loss: 0.4077
Epoch 8/10
250/250 [==============================] - 63s 252ms/step - loss: 0.2554 - val_loss: 0.4077
Epoch 9/10
250/250 [==============================] - 63s 252ms/step - loss: 0.2541 - val_loss: 0.4077
Epoch 10/10
250/250 [==============================] - 63s 252ms/step - l

## Evaluate

Evaluation for question answering requires a significant amount of postprocessing. To avoid taking up too much of your time, this guide skips the evaluation step. The [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) still calculates the evaluation loss during training so you're not completely in the dark about your model's performance.

If have more time and you're interested in how to evaluate your model for question answering, take a look at the [Question answering](https://huggingface.co/course/chapter7/7?fw=pt#postprocessing) chapter from the 🤗 Hugging Face Course!

## Inference

In [158]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [159]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="djbirdsall/qa_model_tutorial")
question_answerer(question=question, context=context)

Some layers from the model checkpoint at djbirdsall/qa_model_tutorial were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_499']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at djbirdsall/qa_model_tutorial and are newly initialized: ['dropout_519']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.7956075072288513, 'start': 93, 'end': 95, 'answer': '13'}

Tokenize the text and return TensorFlow tensors:

In [160]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("djbirdsall/qa_model_tutorial")
inputs = tokenizer(question, context, return_tensors="tf")

Pass your inputs to the model and return the `logits`:

In [161]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("djbirdsall/qa_model_tutorial")
outputs = model(**inputs)

Some layers from the model checkpoint at djbirdsall/qa_model_tutorial were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_499']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at djbirdsall/qa_model_tutorial and are newly initialized: ['dropout_539']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Get the highest probability from the model output for the start and end positions:

In [162]:
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

Decode the predicted tokens to get the answer:

In [163]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'13'

In [164]:
question = ""
while(question != "exit"):
  question = input("Ask a question: ")
  context = input("Now, provide some context: ")
  response = question_answerer(question=question, context=context)
  print("Response:", response['answer'])

Ask a question: How many sheep live on the moon?
Now, provide some context: There are 2 cows, 4 pigs, and 6 sheep. 2 sheep live on the moon.
Response: 2
Ask a question: Can you respond in multiple words?
Now, provide some context: This person is wondering if you have the ability to respond with more than one word.
Response: the ability to respond with more than one word.
Ask a question: Wow. You really are the smartest chat bot I have created so far.
Now, provide some context: An appropriate response to being told you are smart would be thank you, I think I am smart too. It would be rude to say that this person is stinky.
Response: I think I am smart too.
Ask a question: Can God create a boulder so heavy that he himself cannot lift it?
Now, provide some context: The user is inquiring about whether an omnipotent being can self-limit its own power
Response: self-limit its own power
Ask a question: How many purple does the alien condone?
Now, provide some context: Three hundred people dev